In [1]:
import time
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import warnings
warnings.filterwarnings('ignore')

#### 평가문제 1

In [20]:
# 1-1. 데이터 가져오기
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
driver = webdriver.Chrome()
driver.get(url)

In [21]:
# 목록에서 찾기
driver.find_element_by_css_selector('#tab3_1').click()

In [22]:
# 서울
driver.find_element_by_css_selector('#indexedA').click()

In [23]:
# 1-2. 원하는 데이터를 찾고, 하나를 선택해서 원하는 정보 추출하기
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('#minwonTbody > tr')
len(trs)

5

In [24]:
tr = trs[0]
name = tr.find('td').get_text().strip()
name = name.split()[-1]
name

'서울중부경찰서'

In [25]:
addr = tr.select_one('.title').get_text().strip()
addr

'서울특별시 중구 수표로 27'

In [26]:
# 1-3. 반복문을 사용하여 모든 페이지의 데이터 가져오기 및 오류 수정하기
lines = []
for tr in trs:
    name = tr.find('td').get_text().strip().split()[-1]
    addr = tr.select_one('.title').get_text().strip()
    lines.append([name, addr])

In [27]:
lines

[['서울중부경찰서', '서울특별시 중구 수표로 27'],
 ['서울종로경찰서', '서울특별시 종로구 율곡로 46'],
 ['서울남대문경찰서', '서울특별시 중구 한강대로 410'],
 ['서울서대문경찰서', '서울특별시 서대문구 통일로 113'],
 ['서울혜화경찰서', '서울특별시 종로구 창경궁로 112-16']]

In [28]:
for _ in range(6):
    driver.find_element_by_css_selector('.p.next').click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('#minwonTbody > tr')
    for tr in trs:
        name = tr.find('td').get_text().strip().split()[-1]
        addr = tr.select_one('.title').get_text().strip()
        lines.append([name, addr])

In [29]:
driver.close()

In [30]:
# 1-4. 데이터 프레임으로 만들고 저장하기
df = pd.DataFrame(lines, columns=['경찰서명','주소'])
df.tail()

,경찰서명,주소
26,서울노원경찰서,서울특별시 노원구 노원로 283
27,서울방배경찰서,서울특별시 서초구 방배천로 54
28,서울도봉경찰서,서울특별시 도봉구 노해로 403
29,서울수서경찰서,서울특별시 강남구 개포로 617
30,서울은평경찰서,서울특별시 은평구 연서로 365


In [35]:
# 서울서부경찰서, 서울강동경찰서 주소를 수동으로 교체
df[df.경찰서명.isin(["서울서부경찰서","서울강동경찰서"])].index

Int64Index([13, 20], dtype='int64')

In [36]:
df.주소[13] = '서울특별시 은평구 서오릉로 58'
df.주소[20] = '서울 강동구 양재대로96길 35'

In [37]:
df.to_csv('서울경찰서위치.csv', index=False)

#### 평가문제 2

In [17]:
import requests
from urllib.parse import quote

In [18]:
# 2-1. 카카오 로컬 API 사이트에 접속하기
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [19]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [31]:
# 2-2. 반복문을 사용하여 모든 경찰서의 위도, 경도 좌표 구하기
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰서명[i])

서울서부경찰서
서울강동경찰서


In [32]:
df[df.경찰서명.isin(["서울서부경찰서","서울강동경찰서"])].주소

13    서울특별시 은평구 은평로9길 15
20      서울특별시 강동구 성내로 33
Name: 주소, dtype: object

In [33]:
url = f'{local_url}?query={quote("서울특별시 은평구 서오릉로 58")}'
result = requests.get(url, headers=header).json()
result['documents'][0]['x']

'126.923591244576'

In [34]:
url = f'{local_url}?query={quote("서울 강동구 양재대로96길 35")}'
result = requests.get(url, headers=header).json()
result['documents'][0]['x']

'127.139188584154'

In [38]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.경찰서명[i])

In [39]:
# 2-3. 데이터프레임에 위도, 경도 컬럼 추가하기
df['위도'] = lat_list
df['경도'] = lng_list
df.head(3)

,경찰서명,주소,위도,경도
0,서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549


#### 평가문제 3

In [40]:
import folium

In [41]:
# 3-1. 서울 지도를 그릴 것
map = folium.Map(location=[37.5502, 126.982], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        # 3-2. 툴팁에는 경찰서명을 표시하고, 
        tooltip=f'{df.경찰서명[i]}',
        # 3-2. 팝업에는 경찰서 주소를 표시하되 팝업창의 너비를 늘려 가로로 표시할 것
        popup=folium.Popup(df.주소[i], max_width=200),
        # 3-3. 아이콘은 기본 모양이 아닌 것으로 표시
        icon=folium.Icon(icon='glyphicon glyphicon-home')
    ).add_to(map)
# 3-3. 지도 제목을 ‘서울소재 경찰서 위치’로 표시할 것
title_html = '<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>'    
map.get_root().html.add_child(folium.Element(title_html))
map